# Xenium preprocessing

rakaia provides the ability to read and visualize spatial transcriptomic datasets such as Visium and Xenium. For Xenium, rakaia supports the overlay of segmentation masks alongside aggregated transcript counts per cell. To be consistent with the format of other technologies, the transcript counts and segmentation mask need to be imported as separate Anndata and tiff objects, respectively. 

In [1]:
import spatialdata as sd
from spatialdata_io import xenium
import rasterio
from rasterio.features import rasterize
import numpy as np
import tifffile as tiff
import warnings
warnings.filterwarnings('ignore')

# Set the input and zarr output directories to whatever you want
xenium_path = "/home/admin/github/rakaia/sandbox/Xenium/"
zarr_path = "/home/admin/github/rakaia/sandbox/Xenium.zarr/"

We will use the Xenium human melanoma dataset from [here](https://www.10xgenomics.com/datasets/human-skin-preview-data-xenium-human-skin-gene-expression-panel-add-on-1-standard) as an example. Download the **Xenium Output bundle (full)** from the Output and supplemental files section in the link, and extract/unzip the contents to the same destination directory as *xenium_path* above.

We will next read the data into a SpatialData object and pull the transcript information with spatial coordinates as an Anndata object. It is important to note that in the Anndata object, the transcript counts are summarized per cell, and the coordinates provided are the approximate centroid of each cell. This may differ from the 10X Xenium explorer, which may render individual transcripts in the same cell with slightly different coordinates/ z-level depending on the resolution zoom level. 

In [2]:
try:
    sdata = xenium(xenium_path)
    sdata.write(zarr_path)
# if the directory already exists, can pass this step
except ValueError:
    pass

sdata = sd.read_zarr(zarr_path)

# get the transcripts in Anndata format from the table slot
adata = sdata['table']

adata

INFO     reading /home/admin/github/rakaia/sandbox/Xenium/cell_feature_matrix.h5                                   


AnnData object with n_obs × n_vars = 87499 × 382
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'region', 'cell_labels'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatialdata_attrs'
    obsm: 'spatial'

Next, we will extract the cell boundaries for the matching segmentation mask, and infer the coordinate bounds to convert the segmentation polygons into a segmentation array. These bounds will match the transcript arrays to the segmentation mask in rakaia. 

In [3]:
# Get the cell segmentation data frame
cells = sdata.shapes['cell_boundaries']

# get the int bounds to know where the segmentation mask is
x_min, y_min = np.min((adata.obsm['spatial']), axis=0)
x_max, y_max = np.max((adata.obsm['spatial']), axis=0)


transform = rasterio.transform.from_bounds(x_min, y_min, x_max,
                                           y_max, int(x_max - x_min), int(y_max - y_min))

shapes = [(geom, idx) for idx, geom in enumerate(cells.geometry)]

# Set the mask shape to be the same as the transcript bounds in rakaia
mask = rasterize(shapes=shapes, out_shape=(int(y_max - y_min), int(x_max - x_min)),
                 transform=transform, dtype='int32')

# Set the name of the output directory and give the outputs a relevant name
outdir = "/home/admin/rakaia/xenium/"
out_name = "melanoma"


# NOTE: in most instances, the coordinates along the y axis 
# need to be flipped prior to export using np.flip with axis 0
tiff.imwrite(f"{outdir}/{out_name}_mask.tiff", np.flip(mask, axis=0))
adata.write_h5ad(f"{outdir}/{out_name}.h5ad")

The resulting output files can be found in the specified **outdir** and can be imported into rakaia using the corresponding uploader components. 